<a href="https://colab.research.google.com/github/charlotteesharpp/IMLO-project/blob/main/Training1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connecting to google drive in order to save and load the model features

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


All imports needed

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import time

Transformations for loading the data into sets:
* Training: augmentation to improve the accuracy
* Validation and Testing: no augemtation

In [3]:
# Define the transform to preprocess the images for the training data
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the images to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
    transforms.RandomRotation(15),  # Randomly rotate images in the range
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),  #chnage the brightness
    transforms.RandomCrop(224, padding=4, padding_mode='reflect'),  # Randomly crop images padded by 4 pixels on each side
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalise the images
])

#for the test and validation sets
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Load in the data, apply the transformations and place in dataloaders

In [4]:
# Load the dataset (you will need to specify the path to your dataset)
train_dataset = datasets.Flowers102(
    root="data",
    split="train",
    transform=train_transform,
    download=True,
)
test_dataset = datasets.Flowers102(
    root="data",
    split="test",
    transform=test_transform,
    download=True,
)

val_dataset = datasets.Flowers102(
    root="data",
    split="val",
    transform=test_transform,
    download=True,
)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

100%|██████████| 344862509/344862509 [00:09<00:00, 34759907.74it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 1428453.60it/s]


100%|██████████| 14989/14989 [00:00<00:00, 19120566.50it/s]


Define a CNN architecture

In [5]:
class FlowerClassifier(nn.Module):
    def __init__(self):
        super(FlowerClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 28 * 28, 2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, 102)

    def forward(self, x):
        x = self.bn1(torch.relu(self.conv1(x)))
        x = self.pool(x)
        x = self.bn2(torch.relu(self.conv2(x)))
        x = self.pool(x)
        x = self.bn3(torch.relu(self.conv3(x)))
        x = self.pool(x)
        x = x.view(-1, 128 * 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

Initialise the model, loss function, and optimiser

In [6]:
model = FlowerClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

Define training, validating and testing algoirthms

In [7]:
def train_model(model, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{num_epochs}]: Loss: {loss.item():.4f}")

def validate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()
    return val_loss

def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for test_images, test_labels in test_loader:
            test_outputs = model(test_images)
            _, predicted = torch.max(test_outputs, 1)
            total += test_labels.size(0)
            correct += (predicted == test_labels).sum().item()
    accuracy = correct / total
    return accuracy

begin training the model

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FlowerClassifier().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/models/flowersv2"))

<All keys matched successfully>

In [10]:
num_epochs = 20
start_time = time.time()

for epoch in range(num_epochs):
        train_model(model, train_loader, optimizer, criterion, epoch)
        if epoch % 5 == 0:
            val_loss = validate_model(model, val_loader, criterion)
            print(f"Validation Loss at Epoch {epoch}: {val_loss:.4f}")

test_accuracy = test_model(model, test_loader)
print(f"Test Accuracy (top-1): {100 * test_accuracy:.2f}%")
print(f"Finished Training. Total elapsed time: {round((time.time() - start_time) / 60, 1)} min")

Epoch [0/20]: Loss: 1.0859
Validation Loss at Epoch 0: 83.5074
Epoch [1/20]: Loss: 1.4680
Epoch [2/20]: Loss: 0.9521
Epoch [3/20]: Loss: 1.1916
Epoch [4/20]: Loss: 0.9657
Epoch [5/20]: Loss: 1.2613
Validation Loss at Epoch 5: 83.4511
Epoch [6/20]: Loss: 1.1049
Epoch [7/20]: Loss: 1.3579
Epoch [8/20]: Loss: 1.1827
Epoch [9/20]: Loss: 0.9690
Epoch [10/20]: Loss: 1.1070
Validation Loss at Epoch 10: 83.5124
Epoch [11/20]: Loss: 1.1021
Epoch [12/20]: Loss: 0.9636
Epoch [13/20]: Loss: 1.1426
Epoch [14/20]: Loss: 0.8641
Epoch [15/20]: Loss: 1.0321
Validation Loss at Epoch 15: 83.5592
Epoch [16/20]: Loss: 1.0616
Epoch [17/20]: Loss: 0.9204
Epoch [18/20]: Loss: 1.2426
Epoch [19/20]: Loss: 1.4579
Test Accuracy (top-1): 31.39%
Finished Training. Total elapsed time: 105.2 min


Save the model

In [11]:
torch.save(model.state_dict(), "/content/drive/MyDrive/models/flowersv3")
print("Saved PyTorch Model State to path")

Saved PyTorch Model State to path


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FlowerClassifier().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/models/flowersv3"))

<All keys matched successfully>

In [13]:
num_epochs = 20
start_time = time.time()

for epoch in range(num_epochs):
        train_model(model, train_loader, optimizer, criterion, epoch)
        if epoch % 5 == 0:
            val_loss = validate_model(model, val_loader, criterion)
            print(f"Validation Loss at Epoch {epoch}: {val_loss:.4f}")

test_accuracy = test_model(model, test_loader)
print(f"Test Accuracy (top-1): {100 * test_accuracy:.2f}%")
print(f"Finished Training. Total elapsed time: {round((time.time() - start_time) / 60, 1)} min")

Epoch [0/20]: Loss: 1.1113
Validation Loss at Epoch 0: 83.5537
Epoch [1/20]: Loss: 0.8210
Epoch [2/20]: Loss: 1.1691
Epoch [3/20]: Loss: 0.9349
Epoch [4/20]: Loss: 1.0818
Epoch [5/20]: Loss: 1.1262
Validation Loss at Epoch 5: 83.5114
Epoch [6/20]: Loss: 1.1110
Epoch [7/20]: Loss: 0.9651
Epoch [8/20]: Loss: 0.7923
Epoch [9/20]: Loss: 1.2166
Epoch [10/20]: Loss: 1.0721
Validation Loss at Epoch 10: 83.3355
Epoch [11/20]: Loss: 1.1073
Epoch [12/20]: Loss: 1.0591
Epoch [13/20]: Loss: 0.8029
Epoch [14/20]: Loss: 1.0544
Epoch [15/20]: Loss: 0.9996
Validation Loss at Epoch 15: 83.5879
Epoch [16/20]: Loss: 1.2971
Epoch [17/20]: Loss: 1.2495
Epoch [18/20]: Loss: 1.4472
Epoch [19/20]: Loss: 0.9258
Test Accuracy (top-1): 31.73%
Finished Training. Total elapsed time: 105.2 min


In [14]:
torch.save(model.state_dict(), "/content/drive/MyDrive/models/flowersv4")
print("Saved PyTorch Model State to path")

Saved PyTorch Model State to path


In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FlowerClassifier().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/models/flowersv4"))

<All keys matched successfully>

In [16]:
num_epochs = 20
start_time = time.time()

for epoch in range(num_epochs):
        train_model(model, train_loader, optimizer, criterion, epoch)
        if epoch % 5 == 0:
            val_loss = validate_model(model, val_loader, criterion)
            print(f"Validation Loss at Epoch {epoch}: {val_loss:.4f}")

test_accuracy = test_model(model, test_loader)
print(f"Test Accuracy (top-1): {100 * test_accuracy:.2f}%")
print(f"Finished Training. Total elapsed time: {round((time.time() - start_time) / 60, 1)} min")

Epoch [0/20]: Loss: 0.8116
Validation Loss at Epoch 0: 83.4801
Epoch [1/20]: Loss: 1.1655
Epoch [2/20]: Loss: 0.9491
Epoch [3/20]: Loss: 0.7156
Epoch [4/20]: Loss: 1.0034
Epoch [5/20]: Loss: 1.2216
Validation Loss at Epoch 5: 83.5812
Epoch [6/20]: Loss: 1.2560
Epoch [7/20]: Loss: 1.1694
Epoch [8/20]: Loss: 1.5422
Epoch [9/20]: Loss: 0.8634
Epoch [10/20]: Loss: 1.3712
Validation Loss at Epoch 10: 83.3877
Epoch [11/20]: Loss: 1.2201
Epoch [12/20]: Loss: 0.9769
Epoch [13/20]: Loss: 1.1209
Epoch [14/20]: Loss: 1.2657
Epoch [15/20]: Loss: 1.2179
Validation Loss at Epoch 15: 83.3774
Epoch [16/20]: Loss: 1.0934
Epoch [17/20]: Loss: 1.0720
Epoch [18/20]: Loss: 1.0935
Epoch [19/20]: Loss: 1.0336
Test Accuracy (top-1): 31.42%
Finished Training. Total elapsed time: 106.5 min


In [17]:
torch.save(model.state_dict(), "/content/drive/MyDrive/models/flowersv5")
print("Saved PyTorch Model State to path")

Saved PyTorch Model State to path


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FlowerClassifier().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/models/flowersv5"))

<All keys matched successfully>

In [19]:
num_epochs = 10
start_time = time.time()

for epoch in range(num_epochs):
        train_model(model, train_loader, optimizer, criterion, epoch)
        if epoch % 5 == 0:
            val_loss = validate_model(model, val_loader, criterion)
            print(f"Validation Loss at Epoch {epoch}: {val_loss:.4f}")

test_accuracy = test_model(model, test_loader)
print(f"Test Accuracy (top-1): {100 * test_accuracy:.2f}%")
print(f"Finished Training. Total elapsed time: {round((time.time() - start_time) / 60, 1)} min")

Epoch [0/10]: Loss: 1.2072
Validation Loss at Epoch 0: 83.2645
Epoch [1/10]: Loss: 1.1150
Epoch [2/10]: Loss: 0.9857
Epoch [3/10]: Loss: 1.1735
Epoch [4/10]: Loss: 1.0973
Epoch [5/10]: Loss: 1.1349
Validation Loss at Epoch 5: 83.5185
Epoch [6/10]: Loss: 1.0231
Epoch [7/10]: Loss: 1.1342
Epoch [8/10]: Loss: 1.0463
Epoch [9/10]: Loss: 0.9220
Test Accuracy (top-1): 31.63%
Finished Training. Total elapsed time: 58.8 min


In [20]:
torch.save(model.state_dict(), "/content/drive/MyDrive/models/flowersv6")
print("Saved PyTorch Model State to path")

Saved PyTorch Model State to path


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FlowerClassifier().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/models/flowersv6"))

<All keys matched successfully>

In [22]:
num_epochs = 10
start_time = time.time()

for epoch in range(num_epochs):
        train_model(model, train_loader, optimizer, criterion, epoch)
        if epoch % 5 == 0:
            val_loss = validate_model(model, val_loader, criterion)
            print(f"Validation Loss at Epoch {epoch}: {val_loss:.4f}")

test_accuracy = test_model(model, test_loader)
print(f"Test Accuracy (top-1): {100 * test_accuracy:.2f}%")
print(f"Finished Training. Total elapsed time: {round((time.time() - start_time) / 60, 1)} min")

Epoch [0/10]: Loss: 1.0165
Validation Loss at Epoch 0: 83.3024
Epoch [1/10]: Loss: 1.1423
Epoch [2/10]: Loss: 1.0977
Epoch [3/10]: Loss: 1.1195
Epoch [4/10]: Loss: 1.1578
Epoch [5/10]: Loss: 0.9420
Validation Loss at Epoch 5: 83.5065
Epoch [6/10]: Loss: 0.8561
Epoch [7/10]: Loss: 1.1503
Epoch [8/10]: Loss: 1.2404
Epoch [9/10]: Loss: 1.0263
Test Accuracy (top-1): 31.48%
Finished Training. Total elapsed time: 59.5 min


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/models/model_weights")
print("Saved PyTorch Model State to path")

training block 1:
* Test Accuracy (top-1): 31.45%
* Total elapsed time: 86.8 min

training block 2:
* Test Accuracy (top-1): 31.39%
* Total elapsed time: 105.2 min

training block 3:
* Test Accuracy (top-1): 31.73%
* Total elapsed time: 105.2 min


training block 4:
* Test Accuracy (top-1): 31.42%
* Total elapsed time: 106.5 min

training block 5:
* Test Accuracy (top-1): 31.63%
* Total elapsed time: 58.8 min